In [13]:
####### dropout resnet18 vs without dropout
#### 
import torch
import sys
sys.path.append("..")
import numpy as np

CLEAN_PATH = "/home/yifan/experiments/backdoor/resnet18_CIFAR10_target_5/experiment1"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"


ENCODER_DIMS=[512,256,256,256,256,2]
DECODER_DIMS= [2,256,256,256,256,512]
VIS_MODEL_NAME = 'vis'

DEVICE='cuda:0'
########## initulize reference data and target data
from alignment.data_preprocess import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH)
tar_datainit = DataInit(CLEAN_PATH,CLEAN_PATH,TAR_EPOCH)
ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()


from alignment.ReferenceGenerator import ReferenceGenerator
gen = ReferenceGenerator(ref_provider=ref_provider, tar_provider=tar_provider,REF_EPOCH=REF_EPOCH,TAR_EPOCH=TAR_EPOCH,ref_model=ref_model,tar_model=tar_model,DEVICE=DEVICE)

absolute_alignment_indicates,predict_label_diff_indicates,predict_confidence_Diff_indicates,high_distance_indicates = gen.subsetClassify(mes_val_for_diff=18,mes_val_for_same=0.8,conf_val_for_diff=0.3,conf_val_for_same=0.05)


from representationTrans.trans_visualizer_border import visualizer
from singleVis.SingleVisualizationModel import VisModel
from singleVis.projector import TimeVisProjector
model = VisModel(ENCODER_DIMS, DECODER_DIMS)

I = np.eye(512)
projector = TimeVisProjector(vis_model=model, content_path=REF_PATH, vis_model_name=VIS_MODEL_NAME, device="cpu")
vis = visualizer(ref_provider, I,I, np.dot(ref_provider.train_representation(TAR_EPOCH),I), projector, 200,[0,1],'tab10')


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 8299.31it/s]


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 8616.92it/s]


absolute alignment indicates number: 3 label diff indicates number: 508 confidence diff indicates number: 8 high distance number: 10210


In [24]:
X = torch.Tensor(ref_train_data)
Y = torch.Tensor(tar_train_data)

In [15]:
import torch


from sklearn.decomposition import PCA

# Calculate the mean of each dataset
X_mean = torch.mean(X, dim=0)
Y_mean = torch.mean(Y, dim=0)

# Calculate the covariance matrices for X and Y
X_cov = torch.matmul((X - X_mean).T, X - X_mean)
Y_cov = torch.matmul((Y - Y_mean).T, Y - Y_mean)

# Compute the Singular Value Decomposition (SVD) of the covariance matrices
Ux, Sx, Vx = torch.svd(X_cov)
Uy, Sy, Vy = torch.svd(Y_cov)

# Check if the eigenvectors of X and Y are the same
if torch.allclose(Ux, Uy):
    # If the eigenvectors are the same, you can transform X to Y by multiplying X by Vx.T and then Y by Ux.
    A = torch.matmul(Vx.T, Ux)
    X_transformed = torch.matmul(X, A)
    Y_transformed = torch.matmul(Y, Ux)
    print('X can be linearly transformed into Y.')
else:
    # If the eigenvectors are not the same, you can find the closest linear transformation using Principal Component Analysis (PCA).
    pca = PCA(n_components=X.shape[1])
    pca.fit(X.detach().numpy())
    A = torch.matmul(torch.tensor(pca.components_).T, Uy)
    X_transformed = torch.matmul(X, A)
    Y_transformed = torch.matmul(Y, Uy)
    print('X can be approximately linearly transformed into Y.')


X can be approximately linearly transformed into Y.


In [29]:
import numpy as np

def cka(X, Y, n_samples=2000, reg=1e-8):
    """
    Computes the Centered Kernel Alignment (CKA) between two sets of embeddings X and Y.

    Args:
    X: A PyTorch tensor of shape (n_samples, n_features) representing the embeddings of the first set.
    Y: A PyTorch tensor of shape (n_samples, n_features) representing the embeddings of the second set.
    n_samples: The number of samples to use for computing the CKA score. Default is 2000.
    reg: A small regularization term to add to the diagonal of the Gram matrices. Default is 1e-8.

    Returns:
    The CKA score between X and Y.
    """

    # Randomly select a subset of the embeddings
    idx = torch.randperm(X.shape[0])[:n_samples]
    X = X[idx]
    Y = Y[idx]

    # Compute the centered Gram matrices of X and Y
    X_centered = X - torch.mean(X, dim=0, keepdim=True)
    Y_centered = Y - torch.mean(Y, dim=0, keepdim=True)
    X_gram = torch.matmul(X_centered.T, X_centered)
    Y_gram = torch.matmul(Y_centered.T, Y_centered)

    # Apply whitening transformation to the centered Gram matrices
    X_gram_diag = torch.diag(X_gram)
    Y_gram_diag = torch.diag(Y_gram)
    X_gram_sqrt = torch.inverse(torch.sqrt(X_gram + reg * torch.eye(X_gram.shape[0], device=X.device)))
    Y_gram_sqrt = torch.inverse(torch.sqrt(Y_gram + reg * torch.eye(Y_gram.shape[0], device=Y.device)))
    X_whitened = torch.matmul(torch.matmul(X_centered, X_gram_sqrt), torch.matmul(X_gram_sqrt, X_centered.T))
    Y_whitened = torch.matmul(torch.matmul(Y_centered, Y_gram_sqrt), torch.matmul(Y_gram_sqrt, Y_centered.T))

    # Compute the CKA score
    return torch.trace(torch.matmul(X_whitened, Y_whitened)) / torch.sqrt(torch.trace(torch.matmul(X_whitened, X_whitened)) * torch.trace(torch.matmul(Y_whitened, Y_whitened)))


# # Define two sets of embeddings X and Y with the same dimensions
# X = np.random.randn(100, 50)
# Y = np.random.randn(100, 50)

# Check that the SVD of X and Y is not enough to ensure the same CKA score
Ux, Sx, Vx = np.linalg.svd(X)
Uy, Sy, Vy = np.linalg.svd(Y)
if np.allclose(Ux, Uy):
    print("SVD of X and Y is not enough to ensure the same CKA score")

# Compute the CKA score between X and Y
cka_score = cka(X, Y)

# Print the CKA score
print("CKA score between X and Y:", cka_score)
